In [1]:
import numpy as np
import datetime

In [6]:
import torch
import torchvision           
import torchvision.transforms as transforms 
from torchvision import datasets
import helper
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
from collections import OrderedDict
from IPython.display import clear_output
from torch.utils.data import DataLoader

In [7]:
%matplotlib inline
from matplotlib.pyplot import imshow, imsave

In [8]:
MODEL_NAME = 'DNN'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("MODEL_NAME = {}, DEVICE = {}".format(MODEL_NAME, DEVICE))

MODEL_NAME = DNN, DEVICE = cpu


In [10]:
transform = transforms.Compose(
    [transforms.ToTensor(),                               # image to tensor
     transforms.Normalize(mean=(0.1307,), std=(0.3081,))
    ])

In [11]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving data.tgz to data.tgz
User uploaded file "data.tgz" with length 20475226 bytes


In [13]:
!ls
! tar xvfz data.tgz

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
data/train/15/209.JPEG
data/train/15/210.JPEG
data/train/15/211.JPEG
data/train/15/212.JPEG
data/train/15/213.JPEG
data/train/15/214.JPEG
data/train/15/215.JPEG
data/train/15/216.JPEG
data/train/15/217.JPEG
data/train/15/218.JPEG
data/train/15/219.JPEG
data/train/15/220.JPEG
data/train/15/221.JPEG
data/train/15/222.JPEG
data/train/15/223.JPEG
data/train/15/224.JPEG
data/train/15/225.JPEG
data/train/15/226.JPEG
data/train/15/227.JPEG
data/train/15/228.JPEG
data/train/15/229.JPEG
data/train/15/230.JPEG
data/train/15/231.JPEG
data/train/15/232.JPEG
data/train/15/233.JPEG
data/train/15/234.JPEG
data/train/15/235.JPEG
data/train/15/236.JPEG
data/train/15/237.JPEG
data/train/15/238.JPEG
data/train/15/239.JPEG
data/train/15/240.JPEG
data/train/15/241.JPEG
data/train/15/242.JPEG
data/train/15/243.JPEG
data/train/15/244.JPEG
data/train/15/245.JPEG
data/train/15/246.JPEG
data/train/15/247.JPEG
data/train/15/248.JPEG
data/train/15/249.JPEG
data/train/15/250.JPE

In [14]:
!ls
!ls ./data
!ls ./data/test/
!ls ./data/train

data  data.tgz	sample_data
test  train
01  03	05  07	09  11	13  15	17  19	21  23
02  04	06  08	10  12	14  16	18  20	22  24
01  03	05  07	09  11	13  15	17  19	21  23
02  04	06  08	10  12	14  16	18  20	22  24


In [15]:
train_dataset = datasets.ImageFolder(root='./data/train',transform=transform)
test_dataset = datasets.ImageFolder(root='./data/test',transform=transform)

In [16]:
train_loader = DataLoader(dataset = train_dataset, batch_size=32, shuffle = True, drop_last = True)
test_loader = DataLoader(dataset = test_dataset, batch_size=32, shuffle = True, drop_last = True)

In [17]:
# utility function to measure time

import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [23]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.dropout = nn.Dropout()
        self.fc1 = nn.Linear(in_features=512, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.fc3 = nn.Linear(in_features=60, out_features=24)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x.reshape(x.size(0), -1)
        x = self.dropout(x) 
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

model = ConvNet()
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.001)
    

In [24]:
total_step = len(train_loader)
pd_results = []
epochs = 5

for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        out = model(images)
        loss = criterion(out, labels)
        
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        total = labels.size(0)
        preds = torch.max(out.data, 1)[1]
        correct = (preds==labels).sum().item()
        
        if (i+1)%100==0:
            results = OrderedDict()
            results['epoch'] = epoch+1
            results['idx'] = (i+1)
            results['loss'] = loss.item()
            results['accuracy'] = 100.*correct/total
            pd_results.append(results)
            df = pd.DataFrame.from_dict(pd_results, orient='columns')
            clear_output(wait=True)
            display(df)

        i=i+1

,epoch,idx,loss,accuracy
0,1,100,3.032323,25.000
1,1,200,2.871231,15.625
2,1,300,2.444529,15.625
3,2,100,2.540043,25.000
4,2,200,2.084640,37.500
5,2,300,2.315491,21.875
6,3,100,1.850258,40.625
7,3,200,2.584594,40.625
8,3,300,2.341572,28.125
9,4,100,2.361390,43.750


In [25]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        out = model(images)
        preds = torch.max(out.data, 1)[1]
        total += len(labels)
        correct += (preds==labels).sum().item()
    print('Test Accuracy: ', 100.*correct/total)

Test Accuracy:  29.987157534246574
